In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [8]:
Logistic2 = pd.read_csv(r'MetaModel\Logistic_model2OnlyDriverMETAMODEL.csv',sep=",")
RF1 = pd.read_csv(r'MetaModel\model1OnlyConstructeurMETAMODEL.csv',sep=",")
Logistic2_y = pd.read_csv(r'MetaModel\y_train_Logistic.csv',sep=",")
RF1_y = pd.read_csv(r'MetaModel\y_train_RF.csv',sep=",")

In [9]:
print(Logistic2.head())
print(Logistic2_y.head())
print(RF1.head())
print(RF1_y.head())


   raceId  driverId  winner  positionOrder
0      18         1       1              1
1      18         2       0              3
2      18         3       0              0
3      18         4       0              0
4      18         5       0              2
   raceId  driverId  winner
0      18         1       1
1      18         2       0
2      18         3       0
3      18         4       0
4      18         5       0
   raceId  driverId  winner  positionOrder
0      18         1       1              1
1      18         2       0              0
2      18         3       0              3
3      18         4       0              0
4      18         5       0              2
   raceId  driverId  winner
0      18         1       1
1      18         2       0
2      18         3       0
3      18         4       0
4      18         5       0


In [26]:
Data_y = pd.merge(Logistic2_y,RF1_y, on = ['raceId','driverId'])

In [27]:
Data_y = Data_y.drop(columns=('winner_y')).set_index(['raceId','driverId'])
Data_y

winner_x
raceId driverId          
18     1                1
       2                0
       3                0
       4                0
       5                0
...                   ...
1047   841              0
       849              0
       825              0
       850              0
       815              0

[22836 rows x 1 columns]

In [28]:
Data_x = pd.merge(Logistic2, RF1, on=['raceId','driverId']).set_index(['raceId','driverId'])
Data_x

winner_x  positionOrder_x  winner_y  positionOrder_y
raceId driverId                                                      
18     1                1                1         1                1
       2                0                3         0                0
       3                0                0         0                3
       4                0                0         0                0
       5                0                2         0                2
...                   ...              ...       ...              ...
1047   841              0                0         0                0
       849              0                0         0                0
       825              0                0         0                0
       850              0                0         0                0
       815              0                0         0                0

[22836 rows x 4 columns]

In [39]:
LogisticMetaModel = LogisticRegression(random_state=48)
param_grid = {
    'penalty': [ "l2"],
    'max_iter': [5000],
    'solver' :["liblinear"]
}

Grid = GridSearchCV(LogisticMetaModel,param_grid, scoring='recall')
Grid.fit(Data_x, Data_y.values.ravel())
print(Grid.best_params_)

{'max_iter': 5000, 'penalty': 'l2', 'solver': 'liblinear'}


In [40]:
Grid.score(Data_x,Data_y)

0.6549586776859504

In [44]:
Logistic_test = pd.read_csv(r'DOC_csv\Logistic_model2OnlyDriver.csv',sep=",")
RF1_test = pd.read_csv(r'DOC_csv\RF_model1OnlyConstructeur.csv',sep=",")
Data_test = pd.merge(Logistic_test, RF1_test, on=['raceId','driverId']).set_index(['raceId','driverId'])
Data_test

winner_x  positionOrder_x  winner_y  positionOrder_y
raceId driverId                                                      
1052   1                1                1         0                2
       830              0                3         0                3
       822              0                0         1                1
       846              0                0         0                0
       815              0                2         0                0
...                   ...              ...       ...              ...
1073   849              0                0         0                0
       841              0                0         0                0
       847              0                0         0                0
       8                0                0         0                0
       853              0                0         0                0

[440 rows x 4 columns]

In [50]:
y_pred = Data_test.drop(columns=['positionOrder_x','winner_y', 'positionOrder_y'])
y_pred

winner_x
raceId driverId          
1052   1                1
       830              0
       822              0
       846              0
       815              0
...                   ...
1073   849              0
       841              0
       847              0
       8                0
       853              0

[440 rows x 1 columns]

In [52]:
probabilite = Grid.predict_proba(Data_test)
probabilite = pd.DataFrame(probabilite)
probabilite = probabilite.drop(columns=(0))
probabilite = np.array(probabilite)
y_pred.loc[:,'PREpositionOrder'] = probabilite
y_pred

winner_x  PREpositionOrder
raceId driverId                            
1052   1                1          0.018332
       830              0          0.016662
       822              0          0.624272
       846              0          0.015515
       815              0          0.016960
...                   ...               ...
1073   849              0          0.015515
       841              0          0.015515
       847              0          0.015515
       8                0          0.015515
       853              0          0.015515

[440 rows x 2 columns]

In [53]:
Taille = y_pred.index
raceID = Taille.get_level_values('raceId').unique()
PodiumOr0 = []
Top1 = []
for RACE in raceID:
    driverID = y_pred["PREpositionOrder"][RACE].index.get_level_values('driverId')
    array = np.array(y_pred["PREpositionOrder"][RACE])
    ranks = array.argsort()
    nb = len(driverID)
    ranks = nb - ranks.argsort()
    count = 0
    for DRIVER in driverID:
        rang = ranks[count]
        if rang <= 3:
            PodiumOr0.append(ranks[count])
            if rang == 1:
                Top1.append(1)
            else:
                Top1.append(0)
        else:
            PodiumOr0.append(0)
            Top1.append(0)
        count += 1 
    count = 0

print(len(PodiumOr0),len(Top1))

440 440


In [54]:
y_pred.loc[:,'winner'] = Top1
y_pred.loc[:,'positionOrder'] = PodiumOr0
y_pred = y_pred.drop(columns=("PREpositionOrder"))

In [56]:
y_pred = y_pred.drop(columns=['winner_x'])
y_pred

winner  positionOrder
raceId driverId                       
1052   1              0              2
       830            0              0
       822            1              1
       846            0              0
       815            0              3
...                 ...            ...
1073   849            0              0
       841            0              0
       847            0              0
       8              0              0
       853            0              0

[440 rows x 2 columns]

In [57]:
y_pred.to_csv("MetaModel/MetaModelResultat.csv")

resultat : 67.272